# Data Cleaning for Gender IDEAL

Before committing, please re-run the kernel with clear any output to avoid any merge issues with jupyter and github.

## Imports

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
#import altair as alt

pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

Vision Category: Scoring
Vision Weight: 10

In [5]:
dfv = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/data_clean/vision_clean_final.csv", index_col=0)

# Reverse indices so that Company 1A is listed first.
dfv.index = dfv.index.values[::-1]
dfv = dfv[::-1]
dfv

dfv

,Organization,number_of_employees,number_of_years,workforce,Q9_A,Q9_B,Q9_C,Q9_D,Q9_E,Q9_F,Q9_G,Q9_H,Q9_I,Q10_A,Q10_B,Q10_C,Q10_D,Q10_E,Q11_A,Q11_B,Q11_C,Q11_D,Q11_E,Q11_F,Q12_A,Q12_B,Q12_C,Q12_D,Q12_E,Q12_F,Q13_A,Q13_B,Q13_C,Q14,Q14_Other,Q15_A,Q15_B,Q15_C,Q15_D,Q15_E,Q15_F,Q15_G,benchmark_group,Q11_G,Q12_G
0,Company 1A,Fewer than 100 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,"Yes, internal statement made to all employees","Yes, official public/external statement made about organization's commitment to gender equity",No EDI report is published,NaN,Womens Empowerment Principles (UN),NaN,NaN,NaN,NaN,NaN,Gender Equity Now,Finance,NaN,NaN
1,Company 1B,100-249 Employees,15 or more years,At least 80% of employees are Salaried,NaN,Women and people who identify as women,Working mothers,"Women of color, including all races and ethnicities",Women of all types of abilities,Persons of all non-binary gender identities,Women of all sexual orientations,Women of all religions,Women of all socio-economic levels,Conducted a demographic analysis of the current workforce,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,All Hiring Managers,Human Resources Dept,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Finance,NaN,NaN
2,Company 1C,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Hourly,NaN,NaN,NaN,"Women of color, including all races and ethnicities",NaN,NaN,NaN,NaN,NaN,NaN,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,Engagement with all staff on targets and their importance to the organization,Training of all employees who have a direct impact on meeting targets,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,Other,"Prepare report on work on EDI, no gender performance goals specifically called out.",NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
3,Company 1D,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conducted a demographic analysis of the current workforce,Conducted a baseline culture analysis using employee feedback to identify gaps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Technology,NaN,NaN
4,Company 1E,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Established goals regarding gender equity for the whole organization,NaN,NaN,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,Human Resources Dept,NaN,NaN,NaN,NaN,No,NaN,NaN,No EDI report is published,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NonProfit,NaN,NaN
5,Company 1F,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,"N/A, No EDI policy in place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Dedicated team responsible for establishing and tracking progress toward gender equity targets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,No,NaN,NaN,Publish an annual EDI report that includes performance data on gender equity,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Finance,NaN,NaN
6,Company 1G,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Sal

In [8]:
# Questions 1, 2, 9, and 10 have equal weight of 25% of the Leadership score.
w9 = 0.10
w10 = 0.15
w11 = 0.20
w12 = 0.25
w13 = 0.20
w14 = 0.10
w = [w9, w10, w11, w12, w13, w14]

vision_weight = 10

# For each Company row, generate score by weights determined above.
dfv = dfv[['Organization','Q9_A', 'Q9_B', 'Q9_C', 'Q9_D', 'Q9_E', 'Q9_F', 'Q9_G', 'Q9_H', 'Q9_I', 'Q10_A', 'Q10_B', 'Q10_C', 'Q10_D', 'Q10_E', 
           'Q11_A', 'Q11_B', 'Q11_C', 'Q11_D', 'Q11_E', 'Q11_F', 'Q11_G', 'Q12_A', 'Q12_B', 'Q12_C', 'Q12_D', 'Q12_E', 'Q12_F', 'Q12_G',
           'Q13_A', 'Q13_B', 'Q13_C', 'Q14', 'Q14_Other', 'Q15_A', 'Q15_B', 'Q15_C', 'Q15_D', 'Q15_E', 'Q15_F', 'Q15_G']].copy()

# Rows 6 (Q11), 7 (Q12), and 13 (Q11) respectively include 'No Response' in their columns, which we will recode to 'NaN'
dfv = dfv.replace(['No Response'], "NaN")
    
dfv

# Function to apply to each row:
def compute_score2(row):
    
    # Q9:
    # Choose all that apply: Sum
    
    s9 = 0
    categories = 9
    point9 = 0.2
    
    q9a = row['Q9_A']
    q9b = row['Q9_B']
    q9c = row['Q9_C']
    q9d = row['Q9_D']
    q9e = row['Q9_E']
    q9f = row['Q9_F']
    q9g = row['Q9_G']
    q9h = row['Q9_H']
    q9i = row['Q9_I']
  
    if q9a == "N/A, No EDI statement in place" or q9a == "NaN":
        s9 += 0
    if q9b == "Women and people who identify as women":
        s9 += point9
    if q9c == "Working mothers":
        s9 += point9
    if q9d == "Women of color, including all races and ethnicities":
        s9 += point9
    if q9e == "Women of all types of abilities":
        s9 += point9
    if q9f == "Persons of all non-binary gender identities": 
        s9 += point9
    if q9g == "Women of all sexual orientations":
        s9 += point9
    if q9h == "Women of all religions":
        s9 += point9
    if q9i == "Women of all socio-economic levels":
        s9 += point9
    
    if s9 > 1:
        s9 = 1
        
    # Q10: 
    # Conducted a demographic analysis of the current workforce = 20%
    # Conducted a baseline culture analysis using employee feedback to identify gaps = 20%
    # Shared performance on these analyses with all people managers = 20% (?)
    # Established goals regarding gender equity for the whole organization = 20%
    # Developed a timebound roadmap to meet established goals = 20%
    # None = 0% (q10_e excluded, since responses are either NaN or None)
    # Q11_e, _f: description / other / None, do not include in score
    # Sum, no cap

    s10 = 0
    point10 = 0.2
    
    q10a = row['Q10_A']
    q10b = row['Q10_B']
    q10c = row['Q10_C']
    q10d = row['Q10_D']
    
    if q10a == "Conducted a demographic analysis of the current workforce":
        s10 += point10
    if q10b == "Conducted a baseline culture analysis using employee feedback to identify gaps":
        s10 += point10
    if q10c == "Established goals regarding gender equity for the whole organization":
        s10 += point10
    if q10d == "Developed a timebound roadmap to meet established goals":
        s10 += point10
        
    # Q11:
    # Dedicated team responsible for establishing and tracking progress toward gender equity targets = 25%
    # Specific timeline on when targets should be met = 25%
    # Engagement with all staff on targets and their importance to the organization = 25%
    # Training of all employees who have a direct impact on meeting targets = 25%
    # No = 0%
    # Sum, no cap
    
    s11 = 0
    point11 = 0.25
    
    q11a = row['Q11_A']
    q11b = row['Q11_B']
    q11c = row['Q11_C']
    q11d = row['Q11_D']
    
    if q11a == "Dedicated team responsible for establishing and tracking progress toward gender equity targets":
        s11 += point11
    if q11b == "Specific timeline on when targets should be met":
        s11 += point11
    if q11c =="Engagement with all staff on targets and their importance to the organization":
        s11 += point11
    if q11d == "Training of all employees who have a direct impact on meeting targets":
        s11 += point11
    
    # Q12:
    # All Hiring Managers = 50%
    # Human Resources Dept = 50%
    # All Senior Management = 50%
    # C-Suite = 50%
    # N/A or No = 0%
    # Sum, cap at 100%
    
    s12 = 0
    point12 = 0.5
    
    q12a = row['Q12_A']
    q12b = row['Q12_B']
    q12c = row['Q12_C']
    q12d = row['Q12_D']
    
    if q12a == "All Hiring Managers":
        s12 += point12
    if q12b == "Human Resources Dept":
        s12 += point12
    if q12c == "All Senior Management": 
        s12 += point12
    if q12d == "C-Suite":
        s12 += point12
        
    if s12 > 1:
        s12 = 1
        
    # Q13:
    # No = 0%
    # Yes, internal statement made to all employees = 50%
    # Yes, official public/external statement made about organization's commitment to gender equity = 100%
    # Sum, cap at 100%
    
    s13 = 0
    
    #Q13_A: No => skip other scoring for this question
    q13b = row['Q13_B']
    q13c = row['Q13_C']
    
    if q13b == "Yes, internal statement made to all employees":
        s13 += 0.5
    if q13c == "Yes, official public/external statement made about organization's commitment to gender equity":
        s13 += 1
        
    if s13 > 1:
        s13 = 1
    
    # Q14
    # No EDI report is published = 0%
    # Publish an annual EDI report that includes performance data on gender equity = 33%
    # Publish an annual EDI report that includes performance data on gender equity with targets for future performance = 66%
    # Publish an annual EDI report that includes performance data on gender equity with targets for future performance and performance on previously set targets = 100%
    # Single pick (No Sum)
    s14 = 0
    
    q14 = row['Q14']
    
    if q14 == "Publish an annual EDI report that includes performance data on gender equity":
        s14 = 0.33
    if q14 == "Publish an annual EDI report that includes performance data on gender equity with targets for future performance":
        s14 = 0.66
    if q14 == "Publish an annual EDI report that includes performance data on gender equity with targets for future performance and performance on previously set targets":
        s14 = 1
    
    #preliminary score
    s = [s9, s10, s11, s12, s13, s14]
    
    # return (s9 * w9) + (s10 * w10) + (s11 * w11) + (s12 * w12) + (s13 * w13) + (s14 * w14)
    return sum(map(lambda a, b: a * b, s, w))

dfv.loc[:,'vision_prelim_score'] = dfv.apply(compute_score2, axis = 1)
dfv.loc[:,'vision_weighted_score'] = dfv['vision_prelim_score'] * vision_weight

final_scores = pd.DataFrame(dfv, columns=['Organization', 'vision_weighted_score'])
final_scores

,Organization,vision_weighted_score
0,Company 1A,3.80
1,Company 1B,4.60
2,Company 1C,1.50
3,Company 1D,0.60
4,Company 1E,2.05
5,Company 1F,0.83
6,Company 1G,1.30
7,Company 1H,1.40
8,Company 1I,5.70
9,Company 1J,4.10


In [10]:
#Write out to csv
final_scores.to_csv("C:/Users/Admin/Desktop/gender-ideal-main/scores/vision_scores_clean.csv")